# Problem Description

The problem that we will use to demonstrate sequence learning in this tutorial is the [IMDB movie review](http://ai.stanford.edu/~amaas/data/sentiment/) sentiment classification problem. 

  - The data was collected by [Stanford researchers and was used in a 2011 paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) where a split of 50-50 of the data was used for training and test. An accuracy of 88.89% was achieved.
  - Keras provides access to the IMDB dataset built-in. The imdb.load_data() function allows you to load the dataset.
  - Each movie review is a variable sequence of words and the sentiment of each movie review must be classified.
  - We will map each movie review into a real vector domain, a popular technique when working with text called word embedding. 
    - We will map each word onto a 32 length real valued vector. 
    - We will also limit the total number of words that we are interested in modeling to the 5000 most frequent words, and zero out the rest.  


In [1]:
import time

start = time.time()
print('시작시간:', time.ctime(start))

시작시간: Thu Sep 16 02:11:01 2021


In [2]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)


# Simple LSTM for Sequence Classification

## Step 1: Data Preparation

In [3]:
# load the dataset but only keep the top n words, zero the rest

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
# truncate and pad input sequences

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

## Step 2 ~ 4: Build, compile and fit the model.

In [5]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))  # 100 memory units (smart neurons)
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64) 
# The model is fit for only 3 epochs because it quickly overfits the problem. 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 406s 1s/step - loss: 0.4777 - accuracy: 0.7584 - val_loss: 0.3229 - val_accuracy: 0.8676
Epoch 2/3
391/391 [==============================] - 324s 828ms/step - loss: 0.3146 - accuracy: 0.8741 - val_loss: 0.3183 - val_accuracy: 0.8679
Epoch 3/3
391/391 [==============================] - 296s 757ms/step - loss: 0.2898 - ac

## Step 5: Evaluate the model.

In [6]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 84.84%


You can see that this simple LSTM with little tuning achieves near [state-of-the-art results](https://paperswithcode.com/sota/sentiment-analysis-on-imdb) on the IMDB problem.

# LSTM For Sequence Classification With Dropout

In [7]:
# LSTM with Dropout for sequence classification in the IMDB dataset
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))   # newly added
model.add(LSTM(100))
model.add(Dropout(0.2))   # newly added
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64,validation_split=0.2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
313/313 [==============================] - 206s 652ms/step - loss: 0.4990 - accuracy: 0.7422 - val_

We can see dropout having the desired impact on training with a slightly slower trend in convergence and in this case a lower final accuracy. The model could probably use a few more epochs of training and may achieve a higher skill (try it an see).

# LSTM For Sequence Classification With [Dropout and Recurrent Dropout](https://stackoverflow.com/questions/49940280/keras-lstm-dropout-vs-recurrent-dropout)

  - Dropout can be applied to the input and recurrent connections of the memory units with the LSTM precisely and separately.
  - Keras provides this capability with parameters on the LSTM layer, the dropout for configuring the input dropout and recurrent_dropout for configuring the recurrent dropout
  - Recurrent Dropout is a regularization method for recurrent neural networks. Dropout is applied to the updates to LSTM memory cells (or GRU states), i.e. it drops out the input/update gate in LSTM/GRU.

  >
  > dropout: [0,1], Fraction of the units to drop for the linear transformation of the inputs.
  >
  > recurrent_dropout: [0,1],  Fraction of the units to drop for the linear transformation of the recurrent state.
  >



In [8]:
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # newly modified
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 512s 1s/step - loss: 0.4793 - accuracy: 0.7671
Epoch 2/3
391/391 [==============================] - 523s 1s/step - loss: 0.3222 - accuracy: 0.8685
Epoch 3/3
391/391 [==============================] - 594s 2s/step - loss: 0.2520 - accuracy: 0.9004
Accuracy: 87.82%


We can see that the LSTM specific dropout has a more pronounced effect on the convergence of the network than the layer-wise dropout. 

As above, the number of epochs was kept constant and could be increased to see if the skill of the model can be further lifted.

# LSTM and Convolutional Neural Network For Sequence Classification

  - Convolutional neural networks excel at learning the spatial structure in input data.
  - The IMDB review data does have a one-dimensional spatial structure in the sequence of words in reviews and the CNN may be able to pick out invariant features for good and bad sentiment. This learned spatial features may then be learned as sequences by an LSTM layer.
  - We can easily add a one-dimensional CNN and max pooling layers after the Embedding layer which then feed the consolidated features to the LSTM. 
    - We can use a smallish set of 32 features with a small filter length of 3. 
    - The pooling layer can use the standard length of 2 to halve the feature map size.

In [9]:
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))   # newly added
model.add(MaxPooling1D(pool_size=2))    # newly added
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 175s 438ms/step - loss: 0.4236 - accuracy: 0.7943
Epoch 

We can see that we achieve similar results to the first example although with faster training time.

I would expect that even better results could be achieved if this example was further extended to use dropout.

In [10]:
execution_time = time.time() - start
print('실행시간(분): ', execution_time/60)

실행시간(분):  70.17539571126302
